In [41]:
!pip install asterix_decoder

     |████████████████████████████████| 252 kB 28.0 MB/s 
  Created wheel for asterix-decoder: filename=asterix_decoder-0.7.2-cp37-cp37m-linux_x86_64.whl size=934019 sha256=9eccf0b7ae7b739bd93bf3d0c9f4a083c3dacb4d75891c8daaf27b4012ccbb72
  Stored in directory: /root/.cache/pip/wheels/c4/bc/6f/370e3eee05007c09740e3278ea1254896944c8933af08743e5
Successfully built asterix-decoder


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive"

Mounted at /content/gdrive
 20150314_105449.mp4
'AERO(2).pdf'
'Alan V. Oppenheim, Alan S. Willsky, S Hamid Nawab - Señales y sistemas   (1998, Pearson Educación).pdf'
'Burgmuller - 25 Easy and Progressive Studies, op 100.PDF'
'Catherine E. Housecroft_ A. G. Sharpe_ Pilar Gil Ruiz_ José Ignacio Álvarez Galindo - Química Inorganica (escaneado)-Pearson Educación (2006).pdf'
'Colab Notebooks'
'Copia de Copia de Main_Felipe_adaptado.ipynb'
'Factura Monitor Msi.pdf'
'Ludwig van Beethoven - Für Elise, WoO 59.pdf'
'Presentación sin título.gslides'
'Series y Pelis con mi bebé <3.gdoc'
'Solucionario. Señales y Sistemas. Oppenheim.pdf'
'ss laplace.mp4'
 TFG
'Viajes con mi bebé <3.gdoc'


In [ ]:
#CÓDIGO QUE SEPARA MENSAJES EN FUNCIÓN DEL BDS a partir de un ast
import asterix
import pandas as pd
from pandas import ExcelWriter

#Importamos el archivo de datos:
sample_filename = "./gdrive/My Drive/TFG/ArchivosAsterix/200223-cen-080001_PAR1_MODO_S.ast"

#Abrimos el archivo de datos y lo leemos:
with open(sample_filename, "rb") as f:
    data = f.read()
    data = data[0:7000000]
    parsed = asterix.parse(data)
    contador = 1

    #BDS 44:
    ICAO_44 = []
    WindSpeed = []
    WindDirec = []
    Temp = []
    Pressure = []
    Humidity = []
    Turbulence = []
    TS_44 = []
    #BDS 50:
    ICAO_50 = []
    ROLL = []
    TRACK_ANG = []
    TRACK_RATE = []
    GS = []
    TAS = []
    TS_50 = []
    #BDS 60:
    ICAO_60 = []
    MAG_HDG = []
    IAS = []
    MACH = []
    V_RATE_BARO = []
    V_RATE_INS = []
    TS_60 = []

    for packet in parsed:
      if packet['crc'] != '00000000':
        if 'I250' in packet and 'I140' in packet  and 'I220' in packet:
          for i in list(range(len(packet['I250']))):
            item = packet['I250'][i]
            if item['BDS']['val'] == '44':
              ICAO_44.append(packet['I220']['ACAddr']['val'])
              TS_44.append(packet['I140']['ToD']['val'])
              if item['STATUS_WIND']['val'] == 0:
                WindSpeed.append('NONE')
                WindDirec.append('NONE')
              else:
                WindSpeed.append(item['WIND_SPEED']['val'])
                WindDirec.append(item['WIND_DIRECTION']['val'])
              if item['STATUS_STATIC_AIR_TEMPERATURE']['val'] == 0:
                Temp.append('NONE')
              else:
                Temp.append(item['STATIC_AIR_TEMPERATURE']['val'])
              if item['STATUS_AVERAGE_STATIC_PRESSURE']['val'] == 0:
                Pressure.append('NONE')
              else:
                Pressure.append(item['AVERAGE_STATIC_PRESSURE']['val'])
              if item['STATUS_HUMIDITY']['val'] == 0:
                Humidity.append('NONE')
              else:
                Humidity.append(item['HUMIDITY']['val'])
              if item['STATUS_TURBULENCE']['val'] == 0:
                Turbulence.append('NONE')
              else:
                Turbulence.append(item['TURBULENCE']['val'])

            elif item['BDS']['val'] == '50':
              ICAO_50.append(packet['I220']['ACAddr']['val'])
              TS_50.append(packet['I140']['ToD']['val'])
              ROLL.append(item['RA']['val'])
              TRACK_ANG.append(item['TTA']['val'])
              TRACK_RATE.append(item['TAR']['val'])
              GS.append(item['GS']['val'])
              TAS.append(item['TAS']['val'])

            elif item['BDS']['val'] == '60':
              ICAO_60.append(packet['I220']['ACAddr']['val'])
              TS_60.append(packet['I140']['ToD']['val'])
              MAG_HDG.append(item['HDG']['val'])
              IAS.append(item['IAS']['val'])
              MACH.append(item['MACH']['val'])
              V_RATE_BARO.append(item['BAR']['val'])
              V_RATE_INS.append(item['IVV']['val'])

      if contador % 10000 == 0:
          print(round(contador/len(parsed)*100,2),"% | ",contador)

      contador += 1

    #Creamos un dicionario con las listas para cada BDS:
    datos_44 = {'ICAO':ICAO_44,'TimeStamp':TS_44,'WindSpeed':WindSpeed,'WindDirec.':WindDirec, 'Temperature':Temp,'Pressure':Pressure,'Humidity':Humidity,'Turbulence':Turbulence}
    df_44 = pd.DataFrame.from_dict(datos_44, orient='index')
    df_44 = df_44.transpose()

    datos_50 = {'ICAO':ICAO_50,'TimeStamp':TS_50,'Roll':ROLL,'TrackAngle':TRACK_ANG,'TrackRate':TRACK_RATE,'TAS':TAS,'GS':GS}
    df_50 = pd.DataFrame.from_dict(datos_50, orient='index')
    df_50 = df_50.transpose()

    datos_60 = {'ICAO':ICAO_60,'TimeStamp':TS_60,'MagHeading':MAG_HDG,'IAS':IAS,'Mach':MACH,'VerticalRate_Baro':V_RATE_BARO,'VerticalRate_Ins':V_RATE_INS}
    df_60 = pd.DataFrame.from_dict(datos_60, orient='index')
    df_60 = df_60.transpose()

    #Creamos los Excel con nuestros dicionarios:
    writer_44 = ExcelWriter("./gdrive/My Drive/TFG/TablasDeDatos/TABLA_BDS_44.xlsx")
    df_44.to_excel(writer_44, 'BDS_44', index=False)
    writer_44.save()

    writer_50 = ExcelWriter("./gdrive/My Drive/TFG/TablasDeDatos/TABLA_BDS_50.xlsx")
    df_50.to_excel(writer_50, 'BDS_50', index=False)
    writer_50.save()

    writer_60 = ExcelWriter("./gdrive/My Drive/TFG/TablasDeDatos/TABLA_BDS_60.xlsx")
    df_60.to_excel(writer_60, 'BDS_60', index=False)
    writer_60.save()


In [ ]:
#CÓDIGO QUE JUNTA MENSAJES DE BDS DIFERENTES EMITIDOS CON DIFERENCIA < 5 seg
import pandas as pd
from pandas import ExcelWriter
import numpy as np
import io

#Abrimos los archivos de datos:
with open("./gdrive/My Drive/TFG/ArchivosAsterix/BDS/BDS50_08_Completo.txt") as bds_50:
  with open("./gdrive/My Drive/TFG/ArchivosAsterix/BDS/BDS60_08_Completo.txt") as bds_60:
    data50 = io.StringIO(bds_50.read().strip())
    data60 = io.StringIO(bds_60.read().strip())

df_50 = pd.read_csv(data50, sep="\t", header=None,lineterminator="\n")
df_60 = pd.read_csv(data60, sep="\t", header=None,lineterminator="\n")

#Creamos listas para manejar los datos de los DBS:
icao_50 = df_50[0]
ts_50 = df_50[1]
roll = df_50[2]
track_ang = df_50[3]
track_rate = df_50[4]
tas = df_50[5]
gs = df_50[6]

icao_60 = df_60[0]
ts_60 = df_60[1]
hdg = df_60[2]
ias = df_60[3]
mach = df_60[4]
vRate_baro = df_60[5]
vRate_ins = df_60[6]

ICAO = []
TS = []
ROLL = []
TRACK_ANG = []
TRACK_RATE = []
TAS = []
GS = []
HDG = []
IAS = []
MACH = []
V_RATE_BARO = []
V_RATE_INS = []
WS = []
WD = []
TEMP = []
PRES = []

#Eliminamos el primer elemento de las series que son las cabeceras ya que nos sobran, y pasamos a listas
icao_50 = list(icao_50[1:])
ts_50 = list(ts_50[1:])
roll = list(roll[1:])
track_ang = list(track_ang[1:])
track_rate = list(track_rate[1:])
tas = list(tas[1:])
gs = list(gs[1:])

icao_60 = list(icao_60[1:])
ts_60 = list(ts_60[1:])
hdg = list(hdg[1:])
ias = list(ias[1:])
mach = list(mach[1:])
vRate_baro = list(vRate_baro[1:])
vRate_ins = list(vRate_ins[1:])

#convertimos ',' en '.' de TimeStamp para que se identifiquen los elementos como floats y no como str y podemas compararlos en el rango de 5 seg:
ts_50 = [c.replace(',', '.') for c in ts_50]
ts_60 = [c.replace(',', '.') for c in ts_60]
ts_50 = [float(item) for item in ts_50]
ts_60 = [float(item) for item in ts_60]

#Creamos una función de agregar elementos a las listas para reducir y simplificar el código
def añadirElementos(i,j):
  global ICAO,TS,ROLL,TRACK_ANG,TRACK_RATE,TAS,GS,HDG,IAS,MACH,V_RATE_BARO,V_RATE_INS,WS,WD,TEMP,PRES

  ICAO.append(icao_60[i])
  TS.append((ts_60[i]+ts_50[j])/2)
  ROLL.append(roll[j])
  TRACK_ANG.append(track_ang[j])
  TRACK_RATE.append(track_rate[j])
  TAS.append(tas[j])
  GS.append(gs[j])
  HDG.append(hdg[i])
  IAS.append(ias[i])
  MACH.append(mach[i])
  V_RATE_BARO.append(vRate_baro[i])
  V_RATE_INS.append(vRate_ins[i])


#Comparamos los BDS:
for i in list(np.arange(len(icao_60))):
  if 0.5 <= (i/len(icao_60)*100) <= 99.5:
    for j in list(np.arange(int((((i/len(icao_60)*100)-0.5)/100)*len(icao_50)),int((((i/len(icao_60)*100)+0.5)/100)*len(icao_50)))):
      if icao_60[i] == icao_50[j]:
        if abs(ts_50[j]-ts_60[i]) <= 5:
          añadirElementos(i,j)

  elif 0.5 > (i/len(icao_60)*100):
    for j in list(np.arange(0,int((((i/len(icao_60)*100)+0.5)/100)*len(icao_50)))):
      if icao_60[i] == icao_50[j]:
        if abs(ts_50[j]-ts_60[i]) <= 5:
          añadirElementos(i,j)
  else:
    for j in list(np.arange(int((((i/len(icao_60)*100)-0.5)/100)*len(icao_50)),len(icao_50))):
      if icao_60[i] == icao_50[j]:
        if abs(ts_50[j]-ts_60[i]) <= 5:
          añadirElementos(i,j)
      

  if i % 10000 == 0:
    print(round(i/len(icao_60)*100,2),"% | ",i)

print('\n...Creating Data Frame...\n')

#Creamos un dicionario con las listas para cada BDS:
datos = {'ICAO':ICAO,'TimeStamp':TS,'Roll':ROLL,'TrackAngle':TRACK_ANG,'TrackRate':TRACK_RATE,'TAS':TAS,'GS':GS,'MagHeading':HDG,'IAS':IAS,'Mach':MACH,'VerticalRate_Baro':V_RATE_BARO,'VerticalRate_Ins':V_RATE_INS}
df = pd.DataFrame.from_dict(datos, orient='index')
df = df.transpose()

print('\n...Saving Data...\n')

#Creamos los Excel con nuestros dicionarios:
writer = ExcelWriter("./gdrive/My Drive/TFG/TablasDeDatos/BDS_Juntos.xlsx")
df.to_excel(writer, 'BDS-50_60', index=False)
writer.save()

print('100 %')

